# <strong><u>2: Speed and Upsample Notebook |</u></strong>

This notebook is designed for speed alignment and data up-sampling in gesture analysis. It uses Python libraries like Pandas for data management, NumPy for numerical operations, and Matplotlib for plotting.

In this workflow, the notebook reads in keypoint data from a pickled DataFrame and performs operations to align speed and possibly upsample the data.

### <strong>Requirements</strong>

To run this notebook, you will need the following Python packages:

- pandas
- numpy
- scipy
- matplotlib

You can install these packages using pip:
/```shell
    pip install pandas numpy scipy matplotlib
/```

### <strong>Key Variables and Functions</strong>
- `KEYPOINT`: Variable specifying which keypoint (e.g., "right_wrist") to focus on
- `keypoints_df`: DataFrame storing keypoint data



### <strong>Importing Libraries</strong>
- `pandas`: For DataFrame operations and data analysis
- `numpy`: For numerical computations
- `scipy`: For scientific computations, including filtering
- `matplotlib`: For plotting and visualizations


In [9]:
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

### <strong>Choosing The Right Keypoints</strong>

Using this script, we can isolate the keypoints with which we would like to perform the analyses on. Below is the index of Keypoints defined in mediapipe:

<figure>
    <img src="https://developers.google.com/static/mediapipe/images/solutions/pose_landmarks_index.png" width=25% height=25%
         alt="Keypoint Index">
    <figcaption>Mediapipe Pose Keypoint Index</figcaption>
</figure>


Luckily, we have created a dictionary of the keypoints and can now refer to them by their name instead of the number its been asigned.

### <center><strong><u>Available Keypoints:</u></strong></center>
<table>
  <tr>
    <td>right_wrist</td>
    <td>left_wrist</td>
    <td>right_elbow</td>
    <td>left_elbow</td>
    <td>nose</td>
  </tr>
  <tr>
    <td>right_shoulder</td>
    <td>left_shoulder</td>
    <td>right_eye</td>
    <td>left_eye</td>
  </tr>
</table>


In [ ]:
PARTICIPANT = "5003_I_board_one"
KEYPOINT = "right_wrist"

### <strong>Load Data</strong>
In this section, the data is loaded into the notebook for further processing. You can adjust the source file paths as needed.

In [ ]:
# Load keypoint data
keypoints_df = pd.read_csv(f'C:/Users/cosmo/Desktop/Random Scripts/Co-Speech Gesture Automation/Co-Speech-Gesture-Automation/MOTION_TRACKING_FILES/' + PARTICIPANT + '_keypoints.csv')
# Convert keypoints data to DataFrame
keypoints_df = pd.DataFrame(keypoints_df)
keypoints_df = keypoints_df.loc[:, ['time_ms', KEYPOINT + '_x', KEYPOINT + '_y', ]]
# Show keypoint data
keypoints_df

### <strong>Calculate Unsmoothed Speed</strong>
Here, the speed of the right wrist is calculated without any smoothing. Parameters such as the sampling rate can be adjusted.

In [ ]:
# Calculate the unsmoothed speed of the right wrist
keypoints_df['delta_time'] = keypoints_df['time_ms'].diff()
keypoints_df['delta_x'] = keypoints_df[KEYPOINT + '_x'].diff()
keypoints_df['delta_y'] = keypoints_df[KEYPOINT + '_y'].diff()

# Speed = sqrt(delta_x^2 + delta_y^2) / delta_time
keypoints_df['speed_unsmooth'] = np.sqrt(keypoints_df['delta_x']**2 + keypoints_df['delta_y']**2) / (keypoints_df['delta_time'] / 1000)  # in unit/s

### <strong>Apply Savitzky-Golay Filtering</strong>
The Savitzky-Golay filtering method is applied to smooth the speed data. You can adjust the window size and polynomial order.

This code block is used to smooth the values in the `speed_unsmooth` column of a Pandas DataFrame called `keypoints_df`. The first line of code sets any values in the `speed_unsmooth` column that are below the 20th percentile to 0, effectively removing any low-speed outliers from the data. The second line of code applies a Savitzky-Golay filter to the `speed_unsmooth` column with a window size of 9 and a polynomial order of 2, and stores the smoothed values in a new column called `speed_smooth`.

The `np.percentile()` function is used to calculate the 20th percentile of the `speed_unsmooth` column, which is used as the threshold for removing low-speed outliers. The `savgol_filter()` function is used to apply a Savitzky-Golay filter to the `speed_unsmooth` column, which is a type of smoothing filter that can be used to remove noise from data while preserving important features such as peaks and valleys.

Overall, this code block is used to smooth the values in the `speed_unsmooth` column of a DataFrame using a Savitzky-Golay filter, and remove low-speed outliers from the data. Possible ways to improve the code include adding comments to explain the purpose of each line of code, and using more descriptive variable names.

In [ ]:
keypoints_df['speed_smooth'] = [0 if x < np.percentile(keypoints_df['speed_unsmooth'], 20) else x for x in keypoints_df['speed_unsmooth']]
keypoints_df['speed_smooth'] = savgol_filter(keypoints_df['speed_unsmooth'], 9, 2)
keypoints_df

### <strong>Align Speed Data with Pitch Data</strong>
This section aligns the calculated speed data with the pitch data. Any time offsets or scaling factors can be adjusted here.

In [ ]:
# Upsample the speed data to match the 10 ms sample rate of the pitch data
keypoints_df['time_ms'] = pd.to_timedelta(keypoints_df['time_ms'], unit='ms')
keypoints_df = keypoints_df.set_index('time_ms')
keypoints_df_upsampled = keypoints_df.resample('10ms').mean().interpolate(method='linear')

# Convert the datetime index to integer format
keypoints_df_upsampled.index = (keypoints_df_upsampled.index - keypoints_df_upsampled.index[0]).total_seconds() * 1000

# Merge the upsampled speed data with the pitch data
aligned_df = keypoints_df_upsampled

# Drop unnecessary columns
aligned_df = aligned_df.drop(columns=['delta_time', 'delta_x', 'delta_y'])

### <strong>Save Aligned Data as CSV</strong>
Finally, the aligned data is saved as a CSV file. You can specify the destination path for the output file.

In [ ]:
# Save the aligned data as a CSV file
aligned_df['time_ms'] = pd.to_numeric(aligned_df.index)
aligned_df.to_csv(f'C:/Users/cosmo/Desktop/Random Scripts/Co-Speech Gesture Automation/Co-Speech-Gesture-Automation/SPEED_FILES/'+ PARTICIPANT + '_' + KEYPOINT + '_processed_data.csv', index=False)
aligned_df.head()

### <strong>Plot Data</strong>

The `plot()` function is used to plot the unsmoothed and smoothed speed over time. The `label` parameter is used to set the legend label for each line. The `color` parameter is used to set the color of each line.



In [ ]:
# Create a plot
fig, ax1 = plt.subplots(figsize=(15, 6))

# Plot unsmoothed and smoothed speed
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Speed (px/ms)', color='tab:blue')
ax1.plot(aligned_df['time_ms'], aligned_df['speed_unsmooth'], label='Unsmoothed Speed', color='tab:blue')
ax1.plot(aligned_df['time_ms'], aligned_df['speed_smooth'], label='Smoothed Speed', color='tab:cyan')
ax1.tick_params(axis='y', labelcolor='tab:blue')

# Add title and legend
plt.title('Unsmoothed Speed and Smoothed Speed Over Time')
ax1.legend(loc='upper left')

plt.show()